# BOEM Demo

Goal: to be able to search for and handle the read in of ocean datasets easily. The package we've written for this is called `ocean_data_gateway`, and here we show a short demo.

In [1]:
import ocean_data_gateway as odg
import pandas as pd
pd.set_option('display.max_rows', 5)

## Find Data in a Region

Here we will search for data in the Bering Sea region.

In [2]:
kw = {
    "min_lon": -180,
    "max_lon": -158,
    "min_lat": 50,
    "max_lat": 66,
    "min_time": '2021-4-1',
    "max_time": '2021-4-2',
}

### All the servers

Set up search object, `data`, then do an initial metadata search to find the dataset_ids of the relevant datasets. We are searching for all variables currently.

In [3]:
%%time

# setup Data search object
data = odg.Gateway(kw=kw, approach='region')

# find dataset_ids to make sure it works
data.dataset_ids[0][:5]


CPU times: user 5.38 s, sys: 406 ms, total: 5.79 s
Wall time: 1min 24s


['noaa_nos_co_ops_nmta2',
 'noaa_nos_co_ops_9462808',
 'gov_noaa_nws_papb',
 'gov_noaa_water_beta2',
 'gov_noaa_nws_pakk']

The search checked `dataset_ids` for each of 5 readers and found the following number of datasets in them:

In [4]:
for dataset_ids in data.dataset_ids:
    print(len(dataset_ids))

217
174
1
19
0


This search 2 ERDDAP servers, but more can be added by the user, 2 Axiom databases, and any known local files.

### Just one server

Since that search took 1.5 min just for the dataset_ids, let's narrow which databases are checked.

In [5]:
%%time

# setup Data search object
data = odg.Gateway(kw=kw, approach='region', readers=odg.erddap, erddap={'known_server': 'ioos'})

# look up dataset_ids
print(data.dataset_ids[0][:5], len(data.dataset_ids[0]))

['noaa_nos_co_ops_nmta2', 'noaa_nos_co_ops_9462808', 'gov_noaa_nws_papb', 'gov_noaa_water_beta2', 'gov_noaa_nws_pakk'] 217
CPU times: user 20.2 ms, sys: 6.33 ms, total: 26.5 ms
Wall time: 940 ms


In [12]:
%%time
data.meta[0]

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 10 µs


,database,download_url,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,time_coverage_start,time_coverage_end,defaultDataQuery,subsetVariables,keywords,id,infoUrl,institution,featureType,source,sourceUrl,variable names
noaa_nos_co_ops_9459881,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,55.059889,55.059889,-162.326111,-162.326111,2005-08-26T18:00:00Z,2021-05-12T19:00:00Z,"wind_speed_qc_agg,sea_surface_height_amplitude...",NA,NA,12010,https://sensors.ioos.us/#metadata/12010/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://tidesandcurrents.noaa.gov/api/,None
noaa_nos_co_ops_twc2223,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,52.716700,52.716700,-170.150000,-170.150000,2015-05-05T19:16:00Z,2021-05-12T03:49:00Z,sea_surface_height_amplitude_due_to_geocentric...,NA,NA,15515,https://sensors.ioos.us/#metadata/15515/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://sensors.axds.co/api/,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
noaa_nos_co_ops_9462808,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,54.393639,54.393639,-164.745720,-164.745720,2009-04-27T22:00:00Z,2021-05-12T19:00:00Z,"sea_surface_height_above_sea_level_geoid_mllw,...",NA,NA,15485,https://sensors.ioos.us/#metadata/15485/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://tidesandcurrents.noaa.gov/api/,None
noaa_nos_co_ops_nmta2,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,64.500000,64.500000,-165.430000,-165.430000,2015-05-05T12:24:00Z,2021-05-05T13:42:00Z,"air_temperature,wind_speed_of_gust,sea_water_t...",NA,NA,13819,https://sensors.ioos.us/#metadata/13819/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://sensors.axds.co/api/,None


In [7]:
%%time
data_out = data.data[0]()

CPU times: user 735 ms, sys: 61.5 ms, total: 796 ms
Wall time: 16.5 s


In [13]:
data_out['noaa_nos_co_ops_nmta2']

,latitude (degrees_north),longitude (degrees_east),z (m),air_pressure (millibars),air_temperature (degree_Celsius),wind_speed_of_gust (mile.hour-1),wind_speed (m.s-1),wind_from_direction (degrees),station
time (UTC),,,,,,,,,
2021-04-02 00:00:00+00:00,64.5,-165.43,0.0,1024.6,-15.8,10.2900,4.1,100.0,"NMTA2 - 9468756 - Nome, Norton Sound, AK"
2021-04-01 23:54:00+00:00,64.5,-165.43,0.0,1024.6,-15.9,9.1714,3.6,90.0,"NMTA2 - 9468756 - Nome, Norton Sound, AK"
...,...,...,...,...,...,...,...,...,...
2021-04-01 00:06:00+00:00,64.5,-165.43,0.0,1018.1,-14.8,24.1590,8.2,320.0,"NMTA2 - 9468756 - Nome, Norton Sound, AK"
2021-04-01 00:00:00+00:00,64.5,-165.43,0.0,1017.9,-14.8,26.3960,9.8,320.0,"NMTA2 - 9468756 - Nome, Norton Sound, AK"


### One variable in one server

In [18]:
%%time

# setup Data search object
data = odg.Gateway(kw=kw, approach='region', readers=odg.erddap, 
                   erddap={'known_server': 'ioos', 'variables': 'sea_water_temperature'})

# look up dataset_ids
print(data.dataset_ids[0][:5], len(data.dataset_ids[0]))

['noaa_nos_co_ops_9468756', 'noaa_nos_co_ops_9468333', 'noaa_nos_co_ops_nmta2', 'noaa_nos_co_ops_9459450', 'noaa_nos_co_ops_9461380'] 23
CPU times: user 82.7 ms, sys: 10.1 ms, total: 92.7 ms
Wall time: 700 ms


In [19]:
%%time
data.meta[0]

CPU times: user 27.4 ms, sys: 5.19 ms, total: 32.6 ms
Wall time: 946 ms


,database,download_url,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,time_coverage_start,time_coverage_end,defaultDataQuery,subsetVariables,keywords,id,infoUrl,institution,featureType,source,sourceUrl,variable names
noaa_nos_co_ops_9459881,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,55.059889,55.059889,-162.326111,-162.326111,2005-08-26T18:00:00Z,2021-05-12T19:00:00Z,"wind_speed_qc_agg,sea_surface_height_amplitude...",NA,NA,12010,https://sensors.ioos.us/#metadata/12010/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://tidesandcurrents.noaa.gov/api/,[sea_water_temperature]
gov_usgs_waterdata_15304010,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,61.889301,61.889301,-158.156842,-158.156842,2015-05-06T20:00:00Z,2021-05-05T13:45:00Z,"air_temperature,river_discharge,sea_water_elec...",NA,NA,11606,https://sensors.ioos.us/#metadata/11606/station,USGS National Water Information System (NWIS),TimeSeries,NA,https://sensors.axds.co/api/,[sea_water_temperature]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
noaa_nos_co_ops_9468333,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,63.871361,63.871361,-160.784300,-160.784300,2011-06-22T21:00:00Z,2021-05-12T19:00:00Z,"wind_speed_qc_agg,sea_surface_height_amplitude...",NA,NA,15668,https://sensors.ioos.us/#metadata/15668/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://tidesandcurrents.noaa.gov/api/,[sea_water_temperature]
noaa_nos_co_ops_9468756,http://erddap.sensors.ioos.us/erddap,http://erddap.sensors.ioos.us/erddap/tabledap/...,64.494611,64.494611,-165.439639,-165.439639,1992-09-08T11:00:00Z,2021-05-12T19:00:00Z,"wind_speed_qc_agg,sea_surface_height_amplitude...",NA,NA,12017,https://sensors.ioos.us/#metadata/12017/station,NOAA Center for Operational Oceanographic Prod...,TimeSeries,NA,https://tidesandcurrents.noaa.gov/api/,[sea_water_temperature]


In [20]:
%%time
data_out = data.data[0]()

CPU times: user 56.3 ms, sys: 5.96 ms, total: 62.3 ms
Wall time: 1.52 s


In [22]:
data_out['noaa_nos_co_ops_9459450']

,longitude (degrees_east),latitude (degrees_north),sea_water_temperature (degree_Celsius)
time (UTC),,,
2021-04-01 00:00:00+00:00,-160.5043,55.33172,3.8
2021-04-01 00:06:00+00:00,-160.5043,55.33172,3.8
...,...,...,...
2021-04-01 23:54:00+00:00,-160.5043,55.33172,3.6
2021-04-02 00:00:00+00:00,-160.5043,55.33172,3.6


## Use Local Files

Local files can be easily input into the gateway using Python package `intake` under the hood. It is set up to automatically recognize either `csv` or `netcdf` files and be able to read them in.

In [11]:
filenames = ['/Users/kthyng/Downloads/ANIMIDA_III_BeaufortSea_2014-2015/kasper-netcdf/ANIMctd14.nc',
             '/Users/kthyng/Downloads/Harrison_Bay_CTD_MooringData_2014-2015/Harrison_Bay_data/SBE16plus_01604787_2015_08_09_final.csv']

data = odg.Gateway(readers=odg.local, local={'filenames': filenames})

In [12]:
data.meta[0]

,coords,time_coverage_end,variables,lat_variable,geospatial_lat_min,geospatial_lat_max,time_coverage_start,geospatial_lon_min,catalog_dir,download_url,time_variable,geospatial_lon_max,lon_variable
ANIMctd14.nc,"[time, lat, lon, pressure]",2014-08-07T21:35:54.000004381,"[station_name, sal, tem, fluoro, turbidity, PA...",lat,69.850874,71.488255,2014-07-31T15:33:33.999999314,-152.581114,/Users/kthyng/.ocean_data_gateway/catalogs/,/Users/kthyng/Downloads/ANIMIDA_III_BeaufortSe...,time,-141.717438,lon
SBE16plus_01604787_2015_08_09_final.csv,NaN,2015-08-09T06:00:05Z,"[time, latitude, longitude, water_depth, Condu...",NaN,70.6349,70.6349,2014-08-01T12:00:05Z,-150.237,/Users/kthyng/.ocean_data_gateway/catalogs/,/Users/kthyng/Downloads/Harrison_Bay_CTD_Moori...,NaN,-150.237,NaN


In [13]:
data.data[0]()['ANIMctd14.nc']

<xarray.Dataset>
Dimensions:              (nzmax: 1587, profile: 57)
Coordinates:
    time                 (profile) datetime64[ns] 2014-08-07T02:02:34.0000028...
    lat                  (profile) float64 71.27 71.23 71.18 ... 70.45 70.46
    lon                  (profile) float64 -152.2 -152.3 ... -145.8 -145.8
    pressure             (profile, nzmax) float64 2.187 2.399 ... -9.999e+03
Dimensions without coordinates: nzmax, profile
Data variables:
    station_name         (profile) |S12 b'1.01        ' ... b'T-XA        '
    sal                  (profile, nzmax) float64 24.85 24.85 ... -9.999e+03
    tem                  (profile, nzmax) float64 1.625 1.589 ... -9.999e+03
    fluoro               (profile, nzmax) float64 0.6842 0.7452 ... -9.999e+03
    turbidity            (profile, nzmax) float64 0.604 0.6895 ... -9.999e+03
    PAR                  (profile, nzmax) float64 9.596 9.097 ... -9.999e+03
    platform_variable    float64 9.969e+36
    instrument_variable  float64 9.969e+36
    crs                  float64 9.969e+36
Attributes: (12/35)
    Conventions:                CF-1.6
    Metadata_Conventions:       Unidata Dataset Discovery v1.0
    featureType:                profile
    cdm_data_type:              Station
    nodc_template_version:      NODC_NetCDF_Profile_Incomplete_Templete_v1.1
    standard_name_vocabulary:   NetCDF Climate and Forecast(CF) Metadata Conv...
    ...                         ...
    keywords:                   OCEAN TEMPERATURE,SALINITY,TURBIDITY,WATER PR...
    acknowledgement:            Kasper, J., CTD measurements collected from s...
    publisher_name:             Tim Whiteaker
    publisher_email:            whiteaker@utexas.edu
    publisher_url:              http://arcticstudies.org/animida_iii
    license:                    Creative Commons Attribution 3.0 United State...

In [14]:
data.data[0]()['SBE16plus_01604787_2015_08_09_final.csv']

,time,latitude,longitude,water_depth,Conductivity_[S/m],Pressure_[db],Temperature_ITS90_[deg C],Salinity_Practical_[PSU],Voltage0_[volts],Instrument_Time_[juliandays],flag
0,2014-08-01T12:00:05Z,70.6349,-150.237,13.0,2.495646,12.687,-1.4619,31.0905,0.3091,213.500058,0.0
1,2014-08-01T13:00:05Z,70.6349,-150.237,13.0,2.495454,12.699,-1.4595,31.0854,0.3265,213.541725,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8945,2015-08-09T05:00:05Z,70.6349,-150.237,13.0,2.591448,12.777,0.3619,30.5086,0.3873,586.208391,0.0
8946,2015-08-09T06:00:05Z,70.6349,-150.237,13.0,2.585462,12.754,0.2862,30.5062,0.2441,586.250058,0.0
